# Sample Model Inference Agent Walkthrough

This notebook will walk users through setting up a Model Inference Agent that leverages  SageMaker Jumpstart models

This agent utilizes Amazon SageMaker  to deploy the model.

# Pre-requisites

1. Go through the notebook environment setup in the agents_catalog/0-Notebook-environment/ folder

2. Modify and Deploy jumpstart_model.yaml to your AWS account to instantiate a SageMaker endpoint and a lambda function that invokes the endpoint

#### Ensure the latest version of boto3 is shown below

In [1]:
!pip freeze | grep boto3

boto3 @ file:///home/conda/feedstock_root/build_artifacts/boto3_1739930060723/work


#### Load in environment variables to notebook

In [14]:
# Retrieve import path
%store -r IMPORTS_PATH

# Retrieve account info
%store -r account_id
%store -r region

# Retrieve model lists
%store -r agent_foundation_model

#### Retrieve imports environment variable and bring libraries into notebook

In [4]:
%run $IMPORTS_PATH

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Successfully imported necessary libraries into notebook


# Agent Creation
In this section we create the sub-agent

#### Define agent configuration below 

In [19]:
agent_name = 'Model-inference-expert'
agent_description = "Foundation model inference with SageMaker Jumpstart"
agent_instruction = """You are a machine learning expert at deploying and running inference with Amazon SageMaker Jumpstart models. Your primary task is to run model inference and provide relevant insights. Use only the appropriate tools as required by the specific question. 
When providing your response: a. Start with a brief summary of your understanding of the user's query. 
b. Explain the steps you're taking to address the query. Ask for clarifications from the user if required. 
c. Present the results of the model inference.""" 

#### Instantiate agent with the desired configuration

In [20]:
agents = AgentsForAmazonBedrock()

model_inference_agent = agents.create_agent(
    agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    code_interpretation=False,
    verbose=False
)

model_inference_agent

('BLNRH1JISF',
 'TSTALIASID',
 'arn:aws:bedrock:us-east-1:942514891246:agent-alias/BLNRH1JISF/TSTALIASID')

#### Extract useful agent information

In [21]:
model_inference_agent_id = model_inference_agent[0]
model_inference_agent_arn = f"arn:aws:bedrock:{region}:{account_id}:agent/{model_inference_agent_id}"

model_inference_agent_id, model_inference_agent_arn

('BLNRH1JISF', 'arn:aws:bedrock:us-east-1:942514891246:agent/BLNRH1JISF')

#### Define function details for ActionGroup

In [22]:
function_defs = [
    {
      "name": "invoke_model_inference",
      "description": "Run inference with the deployed model",
      "parameters": {
        "input_text": {
          "description": "input text to be sent to the model",
          "required": True,
          "type": "string"
        }
      },
      "requireConfirmation": "DISABLED"
    }
]

#### Attach Lambda function and create ActionGroup
Note: This uses the default Lambda function name "imaging-biomarker-lambda", this could be different in your account so double-check that this function exists and if not change the lambda_function_name in the code below

In [23]:
# Define Lambda func. details, hardcode Lambda function name
model_inference_lambda_function_name = "foundation-model-invoke"  # Change if different in your account
model_inference_lambda_function_arn = f"arn:aws:lambda:{region}:{account_id}:function:{model_inference_lambda_function_name}"
%store model_inference_lambda_function_arn

Stored 'model_inference_lambda_function_arn' (str)


In [24]:
agents.add_action_group_with_lambda(
    agent_name=agent_name,
    lambda_function_name=model_inference_lambda_function_name,
    source_code_file=model_inference_lambda_function_arn,
    agent_action_group_name="Model-inference",
    agent_action_group_description="Actions for running inference with specialized model",
    agent_functions=function_defs,
    verbose=True
)

Creating action group: Model-inference...
Lambda ARN: arn:aws:lambda:us-east-1:942514891246:function:foundation-model-invoke
Agent functions: [{'name': 'invoke_model_inference', 'description': 'Run inference with the deployed model', 'parameters': {'input_text': {'description': 'input text to be sent to the model', 'required': True, 'type': 'string'}}, 'requireConfirmation': 'DISABLED'}]


#### Add resource based policy to Lambda function to allow agent to invoke

In [26]:
lambda_client = boto3.client('lambda', region)

# Define the resource policy statement
policy_statement = {
    "Sid": "AllowBedrockAgentAccess",
    "Effect": "Allow",
    "Principal": {
        "Service": "bedrock.amazonaws.com"
    },
    "Action": "lambda:InvokeFunction",
    "Resource": model_inference_lambda_function_arn,
    "Condition": {
        "ArnEquals": {
            "aws:SourceArn": model_inference_agent_arn
        }
    }
}

try:
    # Get the current policy
    response = lambda_client.get_policy(FunctionName=model_inference_lambda_function_arn)
    current_policy = json.loads(response['Policy'])
    
    # Add the new statement to the existing policy
    current_policy['Statement'].append(policy_statement)
    
except lambda_client.exceptions.ResourceNotFoundException:
    # If there's no existing policy, create a new one
    current_policy = {
        "Version": "2012-10-17",
        "Statement": [policy_statement]
    }

# Convert the policy to JSON string
updated_policy = json.dumps(current_policy)

# Add or update the resource policy
response = lambda_client.add_permission(
    FunctionName=model_inference_lambda_function_arn,
    StatementId="AllowModelInferenceAgentAccess2",
    Action="lambda:InvokeFunction",
    Principal="bedrock.amazonaws.com",
    SourceArn=model_inference_agent_arn
)

print("Resource policy added successfully.")
print("Response:", response)

Resource policy added successfully.
Response: {'ResponseMetadata': {'RequestId': '6a769d24-f058-4706-a7fa-cf4bdfcaf312', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Tue, 25 Mar 2025 17:35:35 GMT', 'content-type': 'application/json', 'content-length': '369', 'connection': 'keep-alive', 'x-amzn-requestid': '6a769d24-f058-4706-a7fa-cf4bdfcaf312'}, 'RetryAttempts': 0}, 'Statement': '{"Sid":"AllowModelInferenceAgentAccess2","Effect":"Allow","Principal":{"Service":"bedrock.amazonaws.com"},"Action":"lambda:InvokeFunction","Resource":"arn:aws:lambda:us-east-1:942514891246:function:foundation-model-invoke","Condition":{"ArnLike":{"AWS:SourceArn":"arn:aws:bedrock:us-east-1:942514891246:agent/BLNRH1JISF"}}}'}


#### Invoke Model Inference Expert Agent Test Alias to see that it answers question properly

In [35]:
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region)

session_id = str(uuid.uuid1())


test_query = "Summarize this text: The quick brown fox jumps over the lazy dog near the river on a sunny day while birds chirp in the trees above."

response = bedrock_agent_runtime_client.invoke_agent(
      inputText=test_query,
      agentId=model_inference_agent_id,
      agentAliasId="TSTALIASID", 
      sessionId=session_id,
      enableTrace=True, 
      endSession=False,
      sessionState={}
)

print("Request sent to Agent:\n{}".format(response))
print("====================")
print("Agent processing query now")
print("====================")

# Initialize an empty string to store the answer
answer = ""

# Iterate through the event stream
for event in response['completion']:
    # Check if the event is a 'chunk' event
    if 'chunk' in event:
        chunk_obj = event['chunk']
        if 'bytes' in chunk_obj:
            # Decode the bytes and append to the answer
            chunk_data = chunk_obj['bytes'].decode('utf-8')
            answer += chunk_data

# Now 'answer' contains the full response from the agent
print("Agent Answer: {}".format(answer))
print("====================")

Request sent to Agent:
{'ResponseMetadata': {'RequestId': 'c67e25e9-103f-453a-807c-e540514c580c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 25 Mar 2025 22:48:17 GMT', 'content-type': 'application/vnd.amazon.eventstream', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amzn-requestid': 'c67e25e9-103f-453a-807c-e540514c580c', 'x-amz-bedrock-agent-session-id': '3de7f6ae-09cb-11f0-b8ae-8a4ca2a7c47b', 'x-amzn-bedrock-agent-content-type': 'application/json'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'sessionId': '3de7f6ae-09cb-11f0-b8ae-8a4ca2a7c47b', 'completion': <botocore.eventstream.EventStream object at 0x7f3e7a9a4310>}
Agent processing query now
Agent Answer: A fox is chasing the dog.


#### Now that agent has been tested via direct invoke, prepare it by creating an alias

In [ ]:
model_inference_agent_alias_id, model_inference_agent_alias_arn = agents.create_agent_alias(
    model_inference_agent[0], 'v1'
)

%store model_inference_agent_alias_arn
model_inference_agent_alias_id, model_inference_agent_alias_arn